# What's this?

This notebook describes an analysis of measurements taken from a Core417I board (the one like [Core 407I](http://www.waveshare.com/wiki/Core407I) but featuring a version of the STM32F4 chip with HW crypto accelerator). The board together with a custom sofware package is available from Riscure as a training target dubbed [Piñata](https://www.riscure.com/product/pinata-training-target/). Since vanilla Piñata software doesn't do HMAC SHA1, I (@ceesb) cut and paste some of ST's reference code. Then, I added a command on the Piñata that feeds back a SHA1 output into the HMAC SHA1 computation so that Piñata does many HMACs per input sent from the PC:

```
i = 0
iterations = MSB_UINT32(readfromserial(4))
input = readfromserial(20)
for i < iterations:
   triggerUP()
   output = HMACSHA1(input)
   triggerDOWN()
   input = SHA1(input)
   i++
writetoserial(output)
```

It can be run from the PC by sending a message with command 0xdf:

```
=>   df <UINT32 MSB iterations> <20 bytes MSB input>
<=   <20 bytes output of last HMAC SHA1 iteration>
```

Send me (@ceeesb) a ping for the binary.

# TL;DR

Attack doesn't perform very well, key of inner SHA cannot be recovered. Outer SHA not tried.

# Acquisition

[Riscure EM probe (LS)](https://www.riscure.com/product/em-probe-station/) over the chip, right in the middle.

Picoscope in rapid block mode, 4096 encryptions per scope fetch.  On receiving the 4096 traces on the PC we compute the inputs and store them in a trace set. Since we know the key, we verify that the last HMAC output of the Piñata matches with what we expect, so we know for sure all traces match the inputs. Obviously the acquisition is done with a Julia script, similar to piposcope.jl in Jlsca's example folder, but send me a ping if you want it.

Acquired > 1M traces of 3900 samples at 1Gs/s in *less than 15 minutes*. 3.5Gb 7-zipped trace set [here](https://drive.google.com/open?id=0B-My9BsChztIM21sdWxWRWRrZGs).

# Attack definition

The attack is the same as described in Section 3.4 in this [paper](https://link.springer.com/chapter/10.1007/978-3-319-25915-4_19) by Belaid and others.

We're going to take a look at SHA1 HMAC using the input. The key itself cannot be recovered since there is no differential data being mixed with the key. The two SHA1 states themselves are sufficient to create arbitrary HMACs. One can also attack the outer SHA1 using an output attack, but this is not shown here (although [Jlsca](https://github.com/Riscure/Jlsca) implements this attack too). We'll only be looking at the inner SHA1.

To explain this attack, we first define some terms:
* a0,b0,c0,d0,e0: the 160-bit SHA1 state we are to recover, in 5 32-bit numbers
* W0 - W3: the attacker controlled and known 32-bit inputs
* T0-T3: the value of T for rounds 0-3
* F0-F3: the output value of the F function for rounds 0-3
* Rot(x,a): rotates x left by a bits
* Ch(a,b,c) = (a & b) XOR (~a & c)

We then roll out first 4 rounds of the inner SHA1 loop in the terms defined above (constants omitted):
```
T0 = Rot(a0, 5) + Ch(b0, c0, d0) + e0 + W0
F0 = Ch(b0, c0, d0)

T1 = Rot(T0, 5) + Ch(a0, Rot(b0, 30), c0) + d0 + W1
F1 = Ch(a0, Rot(b0, 30), c0)

T2 = Rot(T1, 5) + Ch(T0, Rot(a0, 30), Rot(b0, 30)) + c0 + W2
F2 = Ch(T0, Rot(a0, 30), Rot(b0, 30))

T3 = Rot(T2, 5) + Ch(T1, Rot(T0, 30), Rot(a0, 30)) + Rot(b0, 30) + W3
F3 = Ch(T1, Rot(T0, 30), Rot(a0, 30))
```

The SHA1 input attack steps are then:
1. DPA attack on 32-bit modular addition to guess "Rot(a0, 5) + Ch(b0, c0, d0)" and predict "T0" since we know "W0" 
2. DPA attack on 32-bit modular addition to guess "Ch(a0, Rot(b0, 30), c0) + d0" and predict "T1" since we know "W1" and "Rot(T0, 5)" 
3. DPA attack on Ch function to guess "Rot(a0, 30)" and predict "F3" since we know "T1" and "Rot(T0, 30)" 
4. DPA attack on Ch function to guess "Rot(b0, 30)" and predict "F2" since we know "T0" and "Rot(a0, 30)"
5. DPA attack on 32-bit modular addition to guess "c0" and predict "T2" since we know "Rot(T1, 5) + Ch(T0, Rot(a0, 30), Rot(b0, 30))" and "W2" 

These 5 attacks allow us to recover the secret SHA1 state a0,b0,c0,d0,e0.  DPA attacks 1-5 are recovering 32-bit numbers and in order to enumerate the guesses we need to split the attacks up in smaller (typically 8-bits or less) attack so that the key space can be enumerated. How to split up the attacks is described in this [paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.94.7333&rep=rep1&type=pdf) by Lemke, Schramm and Paar. Note that DPA attack n depends on the output of attack n-1 for n > 1. In Jlsca terms we'd call these attack *phases*. For DPA 3 and 4 multiple parts of the secret can be recovered simulteaneously since they're independent. Within a *phase* you can therefore have multiple *targets*. 

In Jlsca, the SHA1 attack is split up in 14 phases and targets as follows:

* Phase 1, target 1: byte 0 of DPA 1
* Phase 2, target 1: byte 1 of DPA 1
* Phase 3, target 1: byte 2 of DPA 1
* Phase 4, target 1: byte 3 of DPA 1
* Phase 5, target 1: byte 0 of DPA 2
* Phase 6, target 1: byte 1 of DPA 2
* Phase 7, target 1: byte 2 of DPA 2
* Phase 8, target 1: byte 3 of DPA 2
* Phase 9, targets 1-8, 8 nibbles of DPA 3. The reason we attack nibbles is that we constructed the Ch target function to take 2 4-bits inputs and produces one 4-output. We can easily conditionally average on 2 4-bit (8-bits) of input, but not much more than that.
* Phase 10, targets 1-4, 4 bytes of DPA 4. The reason we *don't* attack nibbles here is that one of the Ch target function inputs, Rot(b0,30), is a constant. Since it's constant, it doesn't affect the number of averages so we can take 8-bits of the T0 input to Ch and guess an 8-bits output.
* Phase 11, target 1: byte 0 of DPA 5
* Phase 12, target 1: byte 1 of DPA 5
* Phase 13, target 1: byte 2 of DPA 5
* Phase 14, target 1: byte 3 of DPA 5



Note also that this attack only uses the first 12 bytes of the input! (W0, W1 and W2)

Note also that for HMAC we need to perform the SHA1 attack two times, but we don't in this notebook. 

In [ ]:
# add two local workers in addition to the master process
addprocs(2)

# import the necessary stuff on each process
@everywhere begin
    using Jlsca.Trs
    using Jlsca.Sca
    using Jlsca.Sca.sca
    using Jlsca.Sca.hw
    using Jlsca.Sha
    using Jlsca.Align
    using ProgressMeter
    
    import Jlsca.Sca.leak
end

# plotting libs are local only
using PyPlot.plot,PyPlot.figure,PyPlot.title,PyPlot.subplots

using Base.Threads

@printf("#procs:              %d\n", nprocs())
@printf("#workers:            %d\n", nworkers())
@printf("#threads per worker: %d\n", nthreads())

In [ ]:
@everywhere begin
    # for testing if this notebook is working correctly
    emulation = false
    
    if emulation
        trs = InspectorTrace("../jlsca/shatraces/sha1_67452301efcdab8998badcfe10325476c3d2e1f0.trs")
        # first 32 samples contain all the necessary target intemediates
        addSamplePass(trs, x -> x[1:32])
        # convert the intermediates to the necessary leakages for this attack to work
        addSamplePass(trs, x -> vcat(hw.(x), hw.(x .& 0xf), hw.(x .>> 4)))
        # 16 bytes input, 20 bytes output in this trace set 
        inputLength = 16
        # where the peaks are
        rangeOfInterest = 1:200
    else
        trs = InspectorTrace("pinatahmacsha1.trs")
        # a dirty hack to make Jlsca interpret this data as signed (don't change this to types with different sizes, 
        # or bad fail)
        trs.sampleType = Int8
        # only 20 inputs in this trace set, there's no output
        inputLength = 20
        # where the inner SHA1 is (how this region is determined is explained below)
        rangeOfInterest = 900:2100
    end
    
    # used by function correlateDataSamples defined later
    getTrs() = trs
    
    # set to less if you have no time 
    len = 1000 #length(trs)
    interval = 50
end

# Alignment

First things first, we need some alignment.

In [ ]:
# sets up a static align pass (not for the emulation)

if !emulation 
    @everywhere begin
        popSamplePass(trs)

        # how much we are willing to move to find an optimum correlation with the reference
        maxShift = 100
        # where you want the reference to be placed in the result. Setting this to 1 will place it at the beginning.
        referenceOffset = 900
        # how long the reference is
        referenceLength = 600
        # the reference data itself. Can come from anywhere, but here we extract it from one of traces
        reference = trs[1][2][referenceOffset:referenceOffset+referenceLength-1]
        # drop traces that correlate less than this minimum
        corvalMin = 0.0
        # use the FFTW correlator with the above settings
        alignstate = CorrelationAlignFFT(reference, referenceOffset, maxShift)
        # caches the shift and correlation values, so that in a given process it's only computed once for each trace
        alignpass = AlignPass(alignstate, length(trs), corvalMin)

        addSamplePass(trs, alignpass)
    end
end

We'll plot some trace in the same window to show the effect of the alignment.

In [ ]:
# show a few traces in the same window
for i in 1:20
    plot(trs[i][2])
end

In [ ]:
# zoomed in to check the alignment
for i in 1:20
    plot(trs[i][2][rangeOfInterest])
end


# Finding the inner SHA1

Now we need to find where the inner SHA1 is. We'll use input/output correlation. First we need to define some functions to compute this under a known key. We also define the function that does the correlation (parallel and threaded, just for fun).

In [ ]:
# helper functions for the known key analysis

@everywhere begin
    # will contain the secret inner SHA1 state, used for correlating with intermediates under known key scenario
    # (i.e. known key analysis)
    innerstate = sha1init()
    if emulation
        knownkeyinner = [0x00 for i in 1:20]
        knownkeyinner = hex2bytes("67452301efcdab8998badcfe10325476c3d2e1f0")
    else
        # the master key used for HMAC SHA1 on the pinata
        key = hex2bytes("cafebabedeadbeef0001020304050607cafebabedeadbeef")
        # the inner SHA1 key is derived from the master key by a hash
        innerkey = Sha.K0(key) .⊻ 0x36
        update(innerstate, innerkey)
        # the secret state of the inner SHA1, passed as a known key into the attack later
        knownkeyinner = reinterpret(UInt8, map(hton, innerstate.H))
        # will contain the secret inner SHA1 state, used for known key analysis later
        outerstate = sha1init()
        # the outer SHA1 key is derived from the master key by a hash        
        outerkey = Sha.K0(key) .⊻ 0x5c
        update(outerstate, outerkey)
        # the secret state of the outer SHA1
        knownkeyouter = reinterpret(UInt8, map(hton, outerstate.H))
    end

    # computes the output the inner SHA1, used for finding the relevant sample range
    function inneroutput(innerstate::Sha1state, msg::Vector{UInt8})
        state = deepcopy(innerstate)
        update(state, msg)
        return final(state)
    end

    # given the known inner SHA1 key computes the HD between 0,T0 T0,T1 T1,T2, T2,T3, used for spot finding
    function attackableHDT(msg::Vector{UInt8})
        data = zeros(UInt32, 4)
        cnt = Ref(1)
        innerstate2 = deepcopy(innerstate)
        update(innerstate2, msg[1:inputLength])
        final(innerstate2, (x,y) -> x == "T" && cnt.x <= length(data) ? (data[cnt.x] = y; cnt.x += 1) : nothing)
        xor = 0x00000000
        for i in 1:length(data)
            tmp = data[i]
            data[i] ⊻= xor
            xor = tmp
        end
        return map(hton, data)
    end
    
    # given the known inner SHA1 key computes the HD between T0,W0 T1,W1 T2,W2, T3,W3
    # our trace set doesn't leak this though .. used for spot finding
    function attackableHDIn(msg::Vector{UInt8})
        data = zeros(UInt32, 4)
        cnt = Ref(1)
        innerstate2 = deepcopy(innerstate)
        update(innerstate2, msg[1:inputLength])
        final(innerstate2, (x,y) -> x == "T" && cnt.x <= length(data) ? (data[cnt.x] = y; cnt.x += 1) : nothing)
        for i in 1:length(data)
            data[i] ⊻= Sca.getInt(i,msg)
        end
        return map(hton, data)
    end
end


# Helper for computing correlation for known key analysis. We don't define this in the @everwhere macro, since we 
# only need it on the  master process. It uses IncrementalCovarianceTiled which is threaded, and macro @parallel to
# run it on different processes. Used for spot finding
# TODO move this into Jlsca
function correlateDataSamples()
    @everywhere begin 
        ic = IncrementalCovarianceTiled(length(trs[1][1]), length(trs[1][2]))
        getIc() = ic
    end
    
    @sync @parallel for t in 1:len
        trs = getTrs()
        ic = getIc()
        (data, samples) = trs[t]
        if length(data) == 0 || length(samples) == 0
            continue
        end
        add!(ic, data, samples)
    end
        
    
    for w in workers()
        if w != myid()
            add!(ic, @fetchfrom w getIc())
        end
    end
    
    C = getCorr(ic)
    return abs.(C)
end

We first want to determine in time when the inner SHA1 computation is happening. Inner SHA1 consumes the input, the next code block compute the correlation of samples with the HW of the input, and then plot it. We'll see large spikes from around sample 900 onwards.

In [ ]:
# byte-wise HW of input correlated with samples
@everywhere begin
    addDataPass(trs, x -> Sca.hw.(x))
    addSamplePass(trs, x -> abs.(x))
end

@time C1 = correlateDataSamples()

@everywhere begin
    popDataPass(trs)
    popSamplePass(trs)
end

In [ ]:
# overall picture of HW input correlation
plot(C1')
figure()

# zoomed in
plot(C1[:,rangeOfInterest]')
figure()


Now let's find the output of the inner SHA1. We'll correlate with HW of 32-bits at the time to get a bit stronger signal. We'll find two peaks, one around sample 2100 and one near the end at 3500. The last one is (probably) caused by the outer SHA1 overwriting the inner SHA1 output. We'll use 900:2100 as the range for the inner SHA1 computation.

In [ ]:
# byte-wise HW of inner SHA1 output correlated with samples
@everywhere begin
    addDataPass(trs, x -> inneroutput(innerstate,x[1:inputLength]))
    addDataPass(trs, x -> Sca.hw.(reinterpret(UInt32, x)))
    addSamplePass(trs, x -> abs.(x))
end

@time C2 = correlateDataSamples()

@everywhere begin
    popDataPass(trs)
    popDataPass(trs)
    popSamplePass(trs)
end


In [ ]:
# overall picture of HW SHA1 output correlation
plot(C2')
figure()

# zoomed in
plot(C2[:,rangeOfInterest]')
figure()

Now that we know where the inner SHA1 is, we define some functions to run the actual attack. We define a cond avg variant, and a inc cpa variant.

In [ ]:
function allocateCbData(params::DpaAttack)
    phases = numberOfPhases(params.attack)
    leakages = Sca.getNrLeakageFunctions(params.analysis)
    c = Vector{Vector{Vector{AbstractArray{Float64,2}}}}(phases)
    
    for phase in 1:phases
        targets = numberOfTargets(params.attack,phase)
        c[phase] = Vector{Vector{AbstractArray{Float64,2}}}(targets)
        
        for target in 1:targets
            c[phase][target] = Vector{AbstractArray{Float64,2}}(leakages)
        end
    end
    return c
end

@everywhere begin    
    # for saving the final correlation matrices (we get called for all the params.updateInterval, if any)
    function corrCb(cbData, phase::Int,targetOffset::Int,leakageIdx::Int,c::AbstractArray{Float64,2})
        # just save a pointer
        cbData[phase][targetOffset][leakageIdx] = c
    end
    
    type Div <: Leakage 
        a::UInt8
    end
    leak(a::Div, x::UInt8) = div(x,a.a)

    type Zero <: Leakage end
    leak(a::Zero, x::UInt8) = x == 0x0 ? 1 : 0

    type FF <: Leakage end
    leak(a::FF, x::UInt8) = x == 0xff ? 1 : 0

    type ID <: Leakage end
    leak(a::ID, x::UInt8) = x
end

function docondavg()
    attack = Sha1InputAttack()
    analysis = CPA()
    global condavgparams = DpaAttack(attack,analysis)
    condavgparams.scoresCallBack = Nullable((b,c,d,e) -> corrCb(condavgcorrMatrices,b,c,d,e))
    condavgparams.knownKey = Nullable(knownkeyinner)
#     condavgparams.phases = [1,2,3,4]
#     condavgparams.targetOffsets = [1]
    condavgparams.updateInterval = interval
    condavgparams.analysis.leakages= [HW()]
    
    global condavgcorrMatrices = allocateCbData(condavgparams)
    
    @everywhere addSamplePass(trs, x -> abs.(x[rangeOfInterest]))
    
    @everywhere setPostProcessor(trs, CondAvg(SplitByTracesBlock()))
    
    @time sca(DistributedTrace(), condavgparams, 1,len)
    
    @everywhere popSamplePass(trs)
end

function doinccpa()
    attack = Sca.Sha1InputAttack()
    analysis = IncrementalCPA()
    global inccpaparams = DpaAttack(attack,analysis)
#     inccpaparams.scoresCallBack = Nullable((b,c,d,e) -> corrCb(inccpacorrMatrices,b,c,d,e))
    inccpaparams.knownKey = Nullable(knownkeyinner)
#     inccpaparams.phases = [1,2,3,4]
#     inccpaparams.targetOffsets = [1]
    inccpaparams.updateInterval = interval
    inccpaparams.analysis.leakages= [HW()]
        
    global inccpacorrMatrices = allocateCbData(inccpaparams)
    
    @everywhere addSamplePass(trs, x -> abs.(x[rangeOfInterest]))    
    
    @everywhere setPostProcessor(trs, IncrementalCorrelation(SplitByTracesBlock()))
    
    @time sca(DistributedTrace(), inccpaparams, 1,len)
    
    @everywhere popSamplePass(trs)
end


We define some functions for plotting the results ...

In [ ]:
using Jlsca.Sca.getTarget

# TODO move these plotting functions into Jlsca once they're stable

function plotScoresPerSample(corrMatrices,params,phase,target)
    rankData = params.rankData
    totalplots = rankData.nrLeakages
    nrTraces = getNumberOfTraces(rankData,phase)[end]
    (fig,axes) = subplots(nrows=Int(ceil(totalplots/3)), ncols=(totalplots > 3 ? 3 : totalplots), figsize=(8, 4), sharey=false, squeeze=false)
    fig[:suptitle]("Scores per sample for phase $phase, target $target, \"$(getTarget(params,phase,target))\" @ $nrTraces traces, $(params.analysis)")
    kb = getCorrectKey(params,phase,target)
    f = 1
    for leakage in 1:rankData.nrLeakages
        cm = corrMatrices[phase][target][leakage]
        if isdefined(params.analysis, :leakages) 
            leakstr = string(params.analysis.leakages[leakage])
        else 
            leakstr = leakage
        end
        axes[f][:set_title]("$leakstr, (0x$(hex(kb)) in red)")
        (rows,cols) = size(cm)
        for i in 1:cols
            if i != kb+1
                axes[f][:plot](cm[:,i], color="grey")
            end
        end
        axes[f][:plot](cm[:,kb+1], color="red", linewidth=1)
        f += 1
    end
end

function plotAllEvolutions(params)
    for phase in getPhases(params.rankData)
        for target in getTargets(params.rankData, phase)
            plotEvolutions(params,phase,target)
        end
    end
end

function plotEvolutions(params,phase,target)
    rankData = params.rankData
    totalplots = 1 + (rankData.nrLeakages > 1 ? rankData.nrLeakages : 0) + 1 + (rankData.nrLeakages > 1 ? rankData.nrLeakages : 0)
    nrTraces = getNumberOfTraces(rankData,phase)[end]
    (fig,axes) = subplots(nrows=Int(ceil(totalplots/3)), ncols=(totalplots > 3 ? 3 : totalplots), figsize=(8, 4), sharey=false)
    fig[:suptitle]("Evolutions for phase $phase, target $target, \"$(getTarget(params,phase,target))\" @ $nrTraces traces, $(params.analysis)")
    
    kb = getCorrectKey(params,phase,target)
    guesses = getGuesses(rankData,phase,target)
    f = 1
    for leakage in 1:rankData.nrLeakages
        if isdefined(params.analysis, :leakages) 
            leakstr = string(params.analysis.leakages[leakage])
        else 
            leakstr = leakage
        end
        axes[f][:set_ylim]([guesses[1],guesses[end]+2])
        axes[f][:set_title]("rank evolution, $leakstr, for 0x$(hex(kb))")
        axes[f][:plot](getRankingsEvolution(rankData,phase,target,leakage,kb))
        f += 1
    end
    if rankData.nrLeakages > 1
        combined = (rankData.nrLeakages > 1 ? ", $(params.leakageCombinator) of scores," : "")
        axes[f][:set_ylim]([guesses[1],guesses[end]+2])
        axes[f][:set_title]("rank evolution$combined for 0x$(hex(kb))")
        axes[f][:plot](getRankingsEvolution(rankData,phase,target,kb))
        f += 1
    end

    for leakage in 1:rankData.nrLeakages
        if isdefined(params.analysis, :leakages) 
            leakstr = string(params.analysis.leakages[leakage])
        else 
            leakstr = leakage
        end
        for guess in guesses
            if guess != kb
                axes[f][:plot](getScoresEvolution(rankData,phase,target,leakage,guess), color="grey")
            end
        end
        axes[f][:set_title]("score evolution, $leakstr, (0x$(hex(kb)) in red)")
        axes[f][:plot](getScoresEvolution(rankData,phase,target,leakage,kb), color="red", linewidth=1)
        f += 1
    end
    if rankData.nrLeakages > 1
        axes[f][:set_title]("score evolution$combined (0x$(hex(kb)) in red)")
        for guess in guesses
            if guess != kb
                axes[f][:plot](getScoresEvolution(rankData,phase,target,guess), color="grey")
            end
        end
        axes[f][:plot](getScoresEvolution(rankData,phase,target,kb), color="red", linewidth=1)
        f += 1
    end
    fig[:subplots_adjust](hspace=0.3)
#     fig[:subplots_adjust](wspace=0.3)
    figure()
end

# Running the attacks

Now we want to perform the attack described above. First, we'll use conditional averaging to speed things up. We will use Hamming weight as a leakage model, and store the evolution rank and scores, and the final correlation matrix for each attack. We'll then plot the results.

In [ ]:
docondavg()

In [ ]:
plotAllEvolutions(condavgparams)

As you can see, it's not all bad for DPA 2 (phases 5-8) and 5 (phases 11-14). Unfortunately, the very first DPA 1 is not very good at all. Since all these DPA attacks depend on the success of the previous attack, it's not very hopeful. DPA 1 is bad because the difference between T0 and W0 is a constant. For DPA attacks 2 and 5 this is not the case. For example, in DPA 2 target function inputs "W1" and "Rot(T0, 5)" are added to a constant to produce T1. As a result, T1 is not a constant value away from W1, which means less ghost peaks.

Besides the rank and score evolution, we also recorded the (last) score matrix per sample for each phase,target,leakage. For example, let's see where phase 5 target 1 (byte 0 of DPA 2) leaks.

In [ ]:
plotScoresPerSample(condavgcorrMatrices,condavgparams,5,1)

Next, we run the same attack but with incremental correlation. 

In [ ]:
doinccpa()

In [ ]:
plotAllEvolutions(inccpaparams)

That's all, no key! But I should re-run the acquisition and get > 10M traces, or if you're a student and you're bored, you can do it for me.